# 1.)Preparación del modelo y librerías

In [ ]:
%%capture
!pip install mxnet-cu112
!pip install gluoncv

In [ ]:
%%capture

import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
import gluoncv as gcv
import hashlib
from pylab import rcParams
from matplotlib import pyplot as plt
from gluoncv import model_zoo, data, utils
import numpy as np
import os
from pathlib import Path
import pandas as pd

rcParams['figure.figsize'] = 5, 10

In [ ]:
network = gcv.model_zoo.get_model('yolo3_darknet53_coco', pretrained=True,ctx=mx.gpu(0))

100%|██████████| 224190/224190 [00:07<00:00, 28655.34KB/s]


In [ ]:
network.reset_class(classes=['truck', 'person','car',"motorcycle"], reuse_weights=['truck', 'person',"car","motorcycle"])

#2.)Parámetros de detección


In [ ]:
#@title Objetos a ubicar en la imagen
objetos = "Todos" #@param ["equipo pesado", "vehículos", "personas","Todos"]
print('Buscar', objetos)

Buscar Todos


In [ ]:
#@title Probabilidad mínima de que sea el objeto
#@markdown Los modelos de reconocimiento establecen una probabilidad que el objeto
#@markdown detectado corresponda a la etiqueta, favor especificarla


probabilidad= 57 #@param {type:"slider", min:30, max:90, step:1}
print("Probabilidad mínima",probabilidad)

Probabilidad mínima 57


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from os import walk
path="/content/drive/MyDrive/GIZ/Resultados/Frames_iniciales"
imgs =next(walk(path), (None, None, []))[2]

In [ ]:
imgs[0]

'frame8430.jpg'

In [ ]:
def load_image(filepath):
    
    im=image.imread(filepath)
    return im
    

In [ ]:
def transform_image(array):
    norm_image,image=data.transforms.presets.yolo.transform_test(array)
    norm_image = norm_image.as_in_context(mx.gpu(0))
    return norm_image

In [ ]:
def detect(network, data):
    pred=network(data)
    class_ids,scores,bounding_boxes=pred
    return class_ids, scores, bounding_boxes

In [ ]:
#im=load_image(inicial)
#im=transform_image(im)

In [ ]:
#class_ids, scores, bounding_boxes =detect(network,im)

In [ ]:
def count_object(network, class_ids, scores, bounding_boxes, object_label, threshold):
    idx=0
    for i in range(len(network.classes)):
        if network.classes[i]==object_label:
            idx=i
    scores=scores[0]
    class_ids=class_ids[0]
    num_object=0
    for i in range(len(scores)):
        proba=scores[i].astype('float32').asscalar()
        if proba>threshold and class_ids[i].asscalar()==idx:
            num_object+=1
    return num_object

In [ ]:
difir = Path(path)

In [ ]:
Difir1=[]
Difir1=pd.DataFrame(Difir1,columns=("camara", "latitud", "longitud","proyecto","etiqueta","probabilidad","cantidad_detectada","anio","mes","dia","hora","minuto","archivo"))

In [ ]:
for filepath in difir.glob('**/*.jpg'):
  print(filepath)
  im=load_image(filepath)
  im=transform_image(im)
  file=str(filepath)
  threshold=probabilidad/100#0.6
  class_ids, scores, bounding_boxes =detect(network,im)
  for object_label in ["truck","person","car"]:
    count = count_object(network, class_ids, scores, bounding_boxes, object_label,threshold)
    Difir1.loc[len(Difir1)] = ["DIFIR1",9.9322,-84.10,"Prueba_GIZ",object_label,threshold,count,file[67:71],file[72:74],file[75:77],file[78:80],file[81:83],file]
    #print("{} objects of class '{}' detected".format(count, object_label))

In [ ]:
Difir1

,camara,latitud,longitud,proyecto,etiqueta,probabilidad,cantidad_detectada,anio,mes,dia,hora,minuto,archivo
0,DIFIR1,9.9322,-84.1,Prueba_GIZ,truck,0.57,0,g,,,,,/content/drive/MyDrive/GIZ/Resultados/Frames_i...
1,DIFIR1,9.9322,-84.1,Prueba_GIZ,person,0.57,0,g,,,,,/content/drive/MyDrive/GIZ/Resultados/Frames_i...
2,DIFIR1,9.9322,-84.1,Prueba_GIZ,car,0.57,0,g,,,,,/content/drive/MyDrive/GIZ/Resultados/Frames_i...
3,DIFIR1,9.9322,-84.1,Prueba_GIZ,truck,0.57,0,g,,,,,/content/drive/MyDrive/GIZ/Resultados/Frames_i...
4,DIFIR1,9.9322,-84.1,Prueba_GIZ,person,0.57,0,g,,,,,/content/drive/MyDrive/GIZ/Resultados/Frames_i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2545,DIFIR1,9.9322,-84.1,Prueba_GIZ,person,0.57,0,g,,,,,/content/drive/MyDrive/GIZ/Resultados/Frames_i...
2546,DIFIR1,9.9322,-84.1,Prueba_GIZ,car,0.57,0,g,,,,,/content/drive/MyDrive/GIZ/Resultados/Frames_i...
2547,DIFIR1,9.9322,-84.1,Prueba_GIZ,truck,0.57,0,g,,,,,/content/drive/MyDrive/GIZ/Resultados/Frames_i...
2548,DIFIR1,9.9322,-84.1,Prueba_GIZ,person,0.57,0,g,,,,,/content/drive/MyDrive/GIZ/Resultados/Frames_i...


In [ ]:
Difir1.to_excel("cgr.xlsx")